문제1 10점   
입력 데이터:  
다운로드: https://github.com/nongaussian/class-2020-spark/blob/master/Week%207/input.zip  
설명 - 영화DB사이트 (MovieLens)에서 수집된 데이터로서 u.data에는 사용자가 영화에 매긴 평점이 담겨있고  
u.item에는 영화마다 제목, 카테고리 등의 정보가 담겨 있다.  
  
입력파일 1 - u.data  

* 라인마다 사용자의 영화평점이 기록되어 있으며 4개의 속성 "user id \t item id \t rating \t timestamp"가 탭으로 구분되어 작성되어 있다.  
* (즉, split("\t") 로 자를 수 있음)  모두 integer값이다.  
* 한 사람이 여러개의 영화에 평점을 매길 수 있으므로 같은 user id로 시작하는 라인이 복수번 나타날수 있다.  
* item id는 아래 u.item파일에 있는 영화 아이디 (movie id)를 가리킨다 (즉, foreign key)  
* rating은 1에서 5까지 범위이다.  
  
입력파일 2 - u.item  
  
* 라인마다 영화 정보가 기록되어 있으며 "movie id | movie title | release date | video release date |
* IMDb URL | unknown | Action | Adventure | Animation |
* Children's | Comedy | Crime | Documentary | Drama | Fantasy |
* Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
* Thriller | War | Western" 의 속성 들이 파이프라인(|)으로 구분되어 작성되어 있다.
* 그러나 아래 문제를 보면 맨 앞의 두개 속성, movie id와 movie title 만 사용한다.
* movie id는 이 파일 안에서 한번만 나타난다 (즉, primary key)
  
문제.  
5점을 받은 영화 중에서 알파벳 'T'로 시작하는 영화의 개수는?  
'T'는 대문자  
주의: 5점을 받은 영화중 알파벳 'T'로 시작하는 영화를 중복해서 세지 않도록!  
제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오.   
즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.  


In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext("local", "transformation 3")

In [2]:
# 공백으로 구분하여 item id (movie id)와 rating만 슬라이싱하여 data로 추출한다 
data = sc.textFile('u.data').map(lambda x: x.split("\t")[1:3])

In [3]:
# |로 구분하여  movie id 와 movie title만 슬라이싱하여 item으로 추출한다 
item = sc.textFile('u.item').map(lambda x: x.split("|")[0:2])

In [4]:
# cogroup 이용
rdd = item.cogroup(data).mapValues(lambda t: ( list(t[0]), list(t[1]) ))

In [5]:
#  (5점이상, 첫글자 'T')인 movie title 추출  # 중복제거
rdd_filtered=rdd.filter(lambda x : '5' in x[1][1] and x[1][0][0][0:1] == 'T').map(lambda x: x[1][0][0]).map(lambda x: x.strip()).distinct()

In [6]:
rdd_filtered.count()

75

문제 2 10점  
문제.  
사용자들에게 1점도 받고 5점도 받은 적이 있는 영화의 제목의 길이 중 가장 긴 영화제목의 길이는?  
  
동일 영화가 어떤 사용자에게는 1점, 어떤 사용자에게는 5점을 받은 경우를 말함  
힌트: cogroup을 쓸 수도 있고 groupByKey를 쓸 수도 있음.  
제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오.   
즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.  

In [7]:
# 사용자들에게 1점도 받고 5점도 받은 적이 있는 영화
rdd_filtered_rating  = rdd.filter(lambda x : ('5' in x[1][1]) and ('1' in x[1][1]))

In [8]:
# 영화제목 길이로 sortBy(내림차순)해서 처음을 추출, 길이
len(rdd_filtered_rating.map(lambda x: x[1][0][0]).sortBy(lambda x: len(x), False).collect()[0])

81

In [ ]:
sc.stop()